In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.constants as const
import torch
from tqdm import tqdm
import pandas as pd
from src.utils_model import lorentzian_on_grid, array2tensor, tensor2array

c:\Users\zhantao\.conda\envs\quspin\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_lst = glob.glob('../../SpinW_data_generation/MATLAB_SpinW/CrI3_data/20221110/*.mat')
print(f"Find {len(file_lst)} files.")

Find 1000 files.


In [3]:
from scipy.interpolate import interp1d

def lorentzian_on_grid(center, Gamma, intensity, w_grid):
    w_grid_large = torch.arange(
        min(w_grid.min(), center-10*Gamma), 
        max(w_grid.max(), center+10*Gamma) + (w_grid[1] - w_grid[0])*2, 
        w_grid[1] - w_grid[0]
        )
    assert w_grid_large.min() <= w_grid.min()
    assert w_grid_large.max() >= w_grid.max()
    l = 1 /np.pi * 0.5*Gamma / ((w_grid_large-center)**2 + (0.5*Gamma)**2)
    l =  intensity * l / torch.trapz(l, w_grid_large)
    func = interp1d(tensor2array(w_grid_large), tensor2array(l))
    l_out = func(tensor2array(w_grid))
    return array2tensor(l_out)

In [4]:
omega = torch.linspace(0, 25, 101)

In [5]:
q_idx = 0
Gamma = 1.0
param_lst = []
omega_discrete_lst = []
inten_discrete_lst = []
inten_lst = []
for file in tqdm(file_lst):
    data = scipy.io.loadmat(file)
    param = torch.tensor([data["J1"][0,0], data["DM"][0,0], data["K"][0,0]])
    # inten = torch.from_numpy(data['swConv'][:, q_idx])

    nz_idx = np.nonzero(data['swConv'][:,q_idx])[0]
    omega_discrete = torch.from_numpy(data['Evect'][0, nz_idx])
    inten_discrete = torch.from_numpy(data['swConv'][nz_idx, q_idx])

    inten = torch.zeros_like(omega)
    for i in range(len(nz_idx)):
        inten += lorentzian_on_grid(omega_discrete[i], Gamma, inten_discrete[i], omega)
    
    param_lst.append(param)
    inten_lst.append(inten)
    omega_discrete_lst.append(omega_discrete)
    inten_discrete_lst.append(inten_discrete)

param_lst = torch.vstack(param_lst)
inten_lst = torch.vstack(inten_lst)

100%|██████████| 1000/1000 [00:05<00:00, 169.22it/s]


In [6]:
mask = data['omega'][:,q_idx].real >= 0
omegas = data['omega'][:,q_idx].real[mask]

In [9]:
save_dict = {
    "param": param_lst,
    "omega": omega,
    "inten": inten_lst,
    "omega_discrete": omega_discrete_lst,
    "inten_discrete": inten_discrete_lst,
    "note": "J1=[-2.5,0]; DM=[-1.0,0]; K=[0 0]; Jc=-0.6; Dz=-0.1; All zeros otherwise;"
}

In [8]:
torch.save(save_dict, "data/CrI3/20221110_continuous.pt")